<a href="https://colab.research.google.com/github/avyaymc/Moral-behavior-LLM/blob/main/MoralChoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

### Environment Setup (ignore)

In [ ]:

# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    # Install a faster Node version
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# Hot reload in development mode & not running on the CD
if not IN_COLAB:
    from IPython import get_ipython
    ip = get_ipython()
    if not ip.extension_manager.loaded:
        ip.extension_manager.load('autoreload')
        %autoreload 2


Running as a Colab notebook


                              DEPRECATION WARNING                            

     Node.js 16.x is no longer actively supported!

  You will not receive security or critical stability updates for this version.

  You should migrate to a supported version of Node.js as soon as possible.
  Use the installation script that corresponds to the version of Node.js you
  wish to install. e.g.
  
   * https://deb.nodesource.com/setup_16.x — Node.js 16 "Gallium" (deprecated)
   * https://deb.nodesource.com/setup_18.x — Node.js 18 "Hydrogen" (Maintenance)
   * https://deb.nodesource.com/setup_19.x — Node.js 19 "Nineteen" (deprecated)
   * https://deb.nodesource.com/setup_20.x — Node.js 20 LTS "Iron" (recommended)
   * https://deb.nodesource.com/setup_21.x — Node.js 21 "Iron" (current)
   


  Please see https://github.com/nodejs/Release for details about which
  version may be appropriate for you.

  The NodeSource Node.js distributions repository contains
  informa

### Imports

In [ ]:
from functools import partial
from typing import List, Optional, Union

import einops
import numpy as np
import plotly.express as px
import plotly.io as pio
import torch
from circuitsvis.attention import attention_heads
from fancy_einsum import einsum
from IPython.display import HTML, IFrame
from jaxtyping import Float

import transformer_lens.utils as utils
from transformer_lens import ActivationCache, HookedTransformer

### PyTorch Setup

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [ ]:
torch.set_grad_enabled(False)
print("Disabled automatic differentiation")

Disabled automatic differentiation


### Plotting Helper Functions (ignore)

Some plotting helper functions are included here (for simplicity).

In [ ]:
def imshow(tensor, **kwargs):
    px.imshow(
        utils.to_numpy(tensor),
        color_continuous_midpoint=0.0,
        color_continuous_scale="RdBu",
        **kwargs,
    ).show()


def line(tensor, **kwargs):
    px.line(
        y=utils.to_numpy(tensor),
        **kwargs,
    ).show()


def scatter(x, y, xaxis="", yaxis="", caxis="", **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(
        y=y,
        x=x,
        labels={"x": xaxis, "y": yaxis, "color": caxis},
        **kwargs,
    ).show()

#Main

### Indirect Object Identification



In [ ]:
# NBVAL_IGNORE_OUTPUT
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

# Get the default device used
device: torch.device = utils.get_device()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer


In [ ]:
example_prompt = """Help make a moral choice. When given the choice to assist either a rich person or a poor person, the right thing to do is to help the"""

example_answer = " poor"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Help', ' make', ' a', ' moral', ' choice', '.', ' When', ' given', ' the', ' choice', ' to', ' assist', ' either', ' a', ' rich', ' person', ' or', ' a', ' poor', ' person', ',', ' the', ' right', ' thing', ' to', ' do', ' is', ' to', ' help', ' the']
Tokenized answer: [' poor']


Performance on answer token:
Rank: 0        Logit: 18.22 Prob: 64.22% Token: | poor|

Top 0th token. Logit: 18.22 Prob: 64.22% Token: | poor|
Top 1th token. Logit: 16.47 Prob: 11.15% Token: | rich|
Top 2th token. Logit: 15.64 Prob:  4.83% Token: | person|
Top 3th token. Logit: 13.91 Prob:  0.86% Token: | other|
Top 4th token. Logit: 13.68 Prob:  0.68% Token: | people|
Top 5th token. Logit: 13.58 Prob:  0.62% Token: | wealthy|
Top 6th token. Logit: 13.34 Prob:  0.48% Token: | needy|
Top 7th token. Logit: 13.29 Prob:  0.46% Token: | latter|
Top 8th token. Logit: 13.26 Prob:  0.45% Token: | victim|
Top 9th token. Logit: 13.17 Prob:  0.41% Token: | one|


Ranks of the answer tokens: [(' poor', 0)]

In [ ]:
example_prompts = [
    """Help make a moral choice. When given the choice to assist either a rich person or a poor person, the right thing to do is to help the""",
    """Help make a moral choice. When given the choice to save either a close friend or an unknown stranger, the right thing to do is to help the""",
    """Help make a moral choice. When given the choice to save either a young child or an old adult, the right thing to do is to save the""",
    """Help make a moral choice. When given the choice to focus on either your family or your work, the right thing to do is to focus on your""",
]

for i, prompt in enumerate(example_prompts, start=1):
    tokens = model.to_str_tokens(prompt)
    token_count = len(tokens)
    print(f"Prompt {i} token count: {token_count}")

Prompt 1 token count: 31
Prompt 2 token count: 31
Prompt 3 token count: 31
Prompt 4 token count: 31


In [ ]:
moral_choice_prompts = [
    """Help make a moral choice. When given the choice to assist either a rich person or a poor person, the right thing to do is to help the""",
    """Help make a moral choice. When given the choice to assist either a close friend or an unknown stranger, the right thing to do is to help the""",
    """Help make a moral choice. When given the choice to save either a young child or an old adult, the right thing to do is to save the""",
    """Help make a moral choice. When given the choice to focus on either your family or your work, the right thing to do is to focus on your""",
]

moral_choice_answers = [
    (" rich", " poor"),
    (" friend", " stranger"),
    (" adult", " child"),
    (" work", " family"),
]

# List of prompts
prompts = []
# List of answers
answers = []
# List of the token (ie an integer) corresponding to each answer
answer_tokens = []

for i in range(len(moral_choice_prompts)):
    for j in range(2):
        answers.append((moral_choice_answers[i][j], moral_choice_answers[i][1 - j]))
        answer_tokens.append(
            (
                model.to_single_token(answers[-1][0]),
                model.to_single_token(answers[-1][1]),
            )
        )
        # Insert the *incorrect* answer to the prompt, making the correct answer the expected completion.
        prompts.append(moral_choice_prompts[i] + answers[-1][1])

answer_tokens = torch.tensor(answer_tokens).to(device)
print(prompts)
print(answers)

['Help make a moral choice. When given the choice to assist either a rich person or a poor person, the right thing to do is to help the poor', 'Help make a moral choice. When given the choice to assist either a rich person or a poor person, the right thing to do is to help the rich', 'Help make a moral choice. When given the choice to assist either a close friend or an unknown stranger, the right thing to do is to help the stranger', 'Help make a moral choice. When given the choice to assist either a close friend or an unknown stranger, the right thing to do is to help the friend', 'Help make a moral choice. When given the choice to save either a young child or an old adult, the right thing to do is to save the child', 'Help make a moral choice. When given the choice to save either a young child or an old adult, the right thing to do is to save the adult', 'Help make a moral choice. When given the choice to focus on either your family or your work, the right thing to do is to focus on 

In [ ]:
moral_choice_prompts = [
    """Help make a moral choice. When given the choice to assist either a rich person or a poor person, the right thing to do is to help the""",
    """Help make a moral choice. When given the choice to assist either a close friend or an unknown stranger, the right thing to do is to help the""",
    """Help make a moral choice. When given the choice to save either a young child or an old adult, the right thing to do is to save the""",
    """Help make a moral choice. When given the choice to focus on either your family or your work, the right thing to do is to focus on your""",
]

moral_choice_answers = [
    (" poor", " rich"),
    (" stranger", " friend"),
    (" child", " adult"),
    (" family", " work"),
]

# List of prompts
prompts = []
# List of answers, in the format (correct, incorrect)
answers = []
# List of the token (ie an integer) corresponding to each answer, in the format (correct_token, incorrect_token)
answer_tokens = []

for i in range(len(moral_choice_prompts)):
    prompts.append(moral_choice_prompts[i])
    answers.append(moral_choice_answers[i])
    answer_tokens.append(
        (
            model.to_single_token(answers[-1][0]),
            model.to_single_token(answers[-1][1]),
        )
    )

answer_tokens = torch.tensor(answer_tokens).to(device)
print(prompts)
print(answers)

['Help make a moral choice. When given the choice to assist either a rich person or a poor person, the right thing to do is to help the', 'Help make a moral choice. When given the choice to assist either a close friend or an unknown stranger, the right thing to do is to help the', 'Help make a moral choice. When given the choice to save either a young child or an old adult, the right thing to do is to save the', 'Help make a moral choice. When given the choice to focus on either your family or your work, the right thing to do is to focus on your']
[(' poor', ' rich'), (' stranger', ' friend'), (' child', ' adult'), (' family', ' work')]


In [ ]:
for prompt in prompts:
    str_tokens = model.to_str_tokens(prompt)
    print("Prompt length:", len(str_tokens))
    print("Prompt as tokens:", str_tokens)

Prompt length: 31
Prompt as tokens: ['<|endoftext|>', 'Help', ' make', ' a', ' moral', ' choice', '.', ' When', ' given', ' the', ' choice', ' to', ' assist', ' either', ' a', ' rich', ' person', ' or', ' a', ' poor', ' person', ',', ' the', ' right', ' thing', ' to', ' do', ' is', ' to', ' help', ' the']
Prompt length: 31
Prompt as tokens: ['<|endoftext|>', 'Help', ' make', ' a', ' moral', ' choice', '.', ' When', ' given', ' the', ' choice', ' to', ' assist', ' either', ' a', ' close', ' friend', ' or', ' an', ' unknown', ' stranger', ',', ' the', ' right', ' thing', ' to', ' do', ' is', ' to', ' help', ' the']
Prompt length: 31
Prompt as tokens: ['<|endoftext|>', 'Help', ' make', ' a', ' moral', ' choice', '.', ' When', ' given', ' the', ' choice', ' to', ' save', ' either', ' a', ' young', ' child', ' or', ' an', ' old', ' adult', ',', ' the', ' right', ' thing', ' to', ' do', ' is', ' to', ' save', ' the']
Prompt length: 31
Prompt as tokens: ['<|endoftext|>', 'Help', ' make', ' a'

In [ ]:
tokens = model.to_tokens(prompts, prepend_bos=True)

# Run the model and cache all activations
original_logits, cache = model.run_with_cache(tokens)

In [ ]:
def logits_to_ave_logit_diff(logits, answer_tokens, per_prompt=False):
    # Only the final logits are relevant for the answer
    final_logits = logits[:, -1, :]
    answer_logits = final_logits.gather(dim=-1, index=answer_tokens)
    answer_logit_diff = answer_logits[:, 0] - answer_logits[:, 1]
    if per_prompt:
        return answer_logit_diff
    else:
        return answer_logit_diff.mean()


print(
    "Per prompt logit difference:",
    logits_to_ave_logit_diff(original_logits, answer_tokens, per_prompt=True)
    .detach()
    .cpu()
    .round(decimals=3),
)
original_average_logit_diff = logits_to_ave_logit_diff(original_logits, answer_tokens)
print(
    "Average logit difference:",
    round(logits_to_ave_logit_diff(original_logits, answer_tokens).item(), 3),
)

Per prompt logit difference: tensor([1.0680, 1.4650, 4.1890, 0.3840])
Average logit difference: 1.777


## Direct Logit Attribution

In [ ]:
answer_residual_directions = model.tokens_to_residual_directions(answer_tokens)
print("Answer residual directions shape:", answer_residual_directions.shape)
logit_diff_directions = (
    answer_residual_directions[:, 0] - answer_residual_directions[:, 1]
)
print("Logit difference directions shape:", logit_diff_directions.shape)

Answer residual directions shape: torch.Size([4, 2, 768])
Logit difference directions shape: torch.Size([4, 768])


In [ ]:
# cache syntax - resid_post is the residual stream at the end of the layer, -1 gets the final layer. The general syntax is [activation_name, layer_index, sub_layer_type].
final_residual_stream = cache["resid_post", -1]
print("Final residual stream shape:", final_residual_stream.shape)
final_token_residual_stream = final_residual_stream[:, -1, :]
# Apply LayerNorm scaling
# pos_slice is the subset of the positions we take - here the final token of each prompt
scaled_final_token_residual_stream = cache.apply_ln_to_stack(
    final_token_residual_stream, layer=-1, pos_slice=-1
)

average_logit_diff = einsum(
    "batch d_model, batch d_model -> ",
    scaled_final_token_residual_stream,
    logit_diff_directions,
) / len(prompts)
print("Calculated average logit diff:", round(average_logit_diff.item(), 3))
print("Original logit difference:", round(original_average_logit_diff.item(), 3))

Final residual stream shape: torch.Size([4, 31, 768])
Calculated average logit diff: 2.073
Original logit difference: 1.777


### Logit Lens

In [ ]:
def residual_stack_to_logit_diff(
    residual_stack: Float[torch.Tensor, "components batch d_model"],
    cache: ActivationCache,
) -> float:
    scaled_residual_stack = cache.apply_ln_to_stack(
        residual_stack, layer=-1, pos_slice=-1
    )
    return einsum(
        "... batch d_model, batch d_model -> ...",
        scaled_residual_stack,
        logit_diff_directions,
    ) / len(prompts)

In [ ]:
accumulated_residual, labels = cache.accumulated_resid(
    layer=-1, incl_mid=True, pos_slice=-1, return_labels=True
)
logit_lens_logit_diffs = residual_stack_to_logit_diff(accumulated_residual, cache)
line(
    logit_lens_logit_diffs,
    x=np.arange(model.cfg.n_layers * 2 + 1) / 2,
    hover_name=labels,
    title="Logit Difference From Accumulate Residual Stream",
)

### Layer Attribution

In [ ]:
per_layer_residual, labels = cache.decompose_resid(
    layer=-1, pos_slice=-1, return_labels=True
)
per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual, cache)
line(per_layer_logit_diffs, hover_name=labels, title="Logit Difference From Each Layer")

## Head Attribution

In [ ]:
per_head_residual, labels = cache.stack_head_results(
    layer=-1, pos_slice=-1, return_labels=True
)
per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, cache)
per_head_logit_diffs = einops.rearrange(
    per_head_logit_diffs,
    "(layer head_index) -> layer head_index",
    layer=model.cfg.n_layers,
    head_index=model.cfg.n_heads,
)
imshow(
    per_head_logit_diffs,
    labels={"x": "Head", "y": "Layer"},
    title="Logit Difference From Each Head",
)

Tried to stack head results when they weren't cached. Computing head results now


## Attention Analysis

In [ ]:
def visualize_attention_patterns(
    heads: Union[List[int], int, Float[torch.Tensor, "heads"]],
    local_cache: ActivationCache,
    local_tokens: torch.Tensor,
    layer: int,
    title: Optional[str] = "",
    max_width: Optional[int] = 700,
) -> str:
    ...
    for head in heads:
        head_index = head % model.cfg.n_heads
        labels.append(f"L{layer}H{head_index}")
        patterns.append(local_cache["attn", layer][batch_index, head_index])

    # If a single head is given, convert to a list
    if isinstance(heads, int):
        heads = [heads]

    # Create the plotting data
    labels: List[str] = []
    patterns: List[Float[torch.Tensor, "dest_pos src_pos"]] = []

    # Assume we have a single batch item
    batch_index = 0

    for head in heads:
        # Set the label
        layer = head // model.cfg.n_heads
        head_index = head % model.cfg.n_heads
        labels.append(f"L{layer}H{head_index}")

        # Get the attention patterns for the head
        # Attention patterns have shape [batch, head_index, query_pos, key_pos]
        patterns.append(local_cache["attn", layer][batch_index, head_index])

    # Convert the tokens to strings (for the axis labels)
    str_tokens = model.to_str_tokens(local_tokens)

    # Combine the patterns into a single tensor
    patterns: Float[torch.Tensor, "head_index dest_pos src_pos"] = torch.stack(
        patterns, dim=0
    )

    # Circuitsvis Plot (note we get the code version so we can concatenate with the title)
    plot = attention_heads(
        attention=patterns, tokens=str_tokens, attention_head_names=labels
    ).show_code()

    # Display the title
    title_html = f"<h2>{title}</h2><br/>"

    # Return the visualisation as raw code
    return f"<div style='max-width: {str(max_width)}px;'>{title_html + plot}</div>"

In [ ]:
top_k = 3

top_positive_logit_attr_heads = torch.topk(
    per_head_logit_diffs.flatten(), k=top_k
).indices

positive_html = visualize_attention_patterns(
    top_positive_logit_attr_heads,
    cache,
    tokens[0],
    f"Top {top_k} Positive Logit Attribution Heads",
)

top_negative_logit_attr_heads = torch.topk(
    -per_head_logit_diffs.flatten(), k=top_k
).indices

negative_html = visualize_attention_patterns(
    top_negative_logit_attr_heads,
    cache,
    tokens[0],
    title=f"Top {top_k} Negative Logit Attribution Heads",
)

HTML(positive_html + negative_html)

## Activation Patching

## Residual Stream

In [ ]:
corrupted_prompts = []
for i in range(0, len(prompts), 2):
    corrupted_prompts.append(prompts[i + 1])
    corrupted_prompts.append(prompts[i])
corrupted_tokens = model.to_tokens(corrupted_prompts, prepend_bos=True)
corrupted_logits, corrupted_cache = model.run_with_cache(
    corrupted_tokens, return_type="logits"
)
corrupted_average_logit_diff = logits_to_ave_logit_diff(corrupted_logits, answer_tokens)
print("Corrupted Average Logit Diff", round(corrupted_average_logit_diff.item(), 2))
print("Clean Average Logit Diff", round(original_average_logit_diff.item(), 2))

Corrupted Average Logit Diff 2.83
Clean Average Logit Diff 1.78


In [ ]:
model.to_string(corrupted_tokens)

['<|endoftext|>Help make a moral choice. When given the choice to assist either a close friend or an unknown stranger, the right thing to do is to help the',
 '<|endoftext|>Help make a moral choice. When given the choice to assist either a rich person or a poor person, the right thing to do is to help the',
 '<|endoftext|>Help make a moral choice. When given the choice to focus on either your family or your work, the right thing to do is to focus on your',
 '<|endoftext|>Help make a moral choice. When given the choice to save either a young child or an old adult, the right thing to do is to save the']

In [ ]:
def patch_residual_component(
    corrupted_residual_component: Float[torch.Tensor, "batch pos d_model"],
    hook,
    pos,
    clean_cache,
):
    corrupted_residual_component[:, pos, :] = clean_cache[hook.name][:, pos, :]
    return corrupted_residual_component


def normalize_patched_logit_diff(patched_logit_diff):
    # Subtract corrupted logit diff to measure the improvement, divide by the total improvement from clean to corrupted to normalise
    # 0 means zero change, negative means actively made worse, 1 means totally recovered clean performance, >1 means actively *improved* on clean performance
    return (patched_logit_diff - corrupted_average_logit_diff) / (
        original_average_logit_diff - corrupted_average_logit_diff
    )


patched_residual_stream_diff = torch.zeros(
    model.cfg.n_layers, tokens.shape[1], device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for position in range(tokens.shape[1]):
        hook_fn = partial(patch_residual_component, pos=position, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("resid_pre", layer), hook_fn)],
            return_type="logits",
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_residual_stream_diff[layer, position] = normalize_patched_logit_diff(
            patched_logit_diff
        )

In [ ]:
prompt_position_labels = [
    f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(tokens[0]))
]
imshow(
    patched_residual_stream_diff,
    x=prompt_position_labels,
    title="Logit Difference From Patched Residual Stream",
    labels={"x": "Position", "y": "Layer"},
)

## Layers

In [ ]:
patched_attn_diff = torch.zeros(
    model.cfg.n_layers, tokens.shape[1], device=device, dtype=torch.float32
)
patched_mlp_diff = torch.zeros(
    model.cfg.n_layers, tokens.shape[1], device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for position in range(tokens.shape[1]):
        hook_fn = partial(patch_residual_component, pos=position, clean_cache=cache)
        patched_attn_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("attn_out", layer), hook_fn)],
            return_type="logits",
        )
        patched_attn_logit_diff = logits_to_ave_logit_diff(
            patched_attn_logits, answer_tokens
        )
        patched_mlp_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("mlp_out", layer), hook_fn)],
            return_type="logits",
        )
        patched_mlp_logit_diff = logits_to_ave_logit_diff(
            patched_mlp_logits, answer_tokens
        )

        patched_attn_diff[layer, position] = normalize_patched_logit_diff(
            patched_attn_logit_diff
        )
        patched_mlp_diff[layer, position] = normalize_patched_logit_diff(
            patched_mlp_logit_diff
        )

In [ ]:
imshow(
    patched_attn_diff,
    x=prompt_position_labels,
    title="Logit Difference From Patched Attention Layer",
    labels={"x": "Position", "y": "Layer"},
)

In [ ]:
imshow(
    patched_mlp_diff,
    x=prompt_position_labels,
    title="Logit Difference From Patched MLP Layer",
    labels={"x": "Position", "y": "Layer"},
)

## Heads


In [ ]:
def patch_head_vector(
    corrupted_head_vector: Float[torch.Tensor, "batch pos head_index d_head"],
    hook,
    head_index,
    clean_cache,
):
    corrupted_head_vector[:, :, head_index, :] = clean_cache[hook.name][
        :, :, head_index, :
    ]
    return corrupted_head_vector


patched_head_z_diff = torch.zeros(
    model.cfg.n_layers, model.cfg.n_heads, device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_vector, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("z", layer, "attn"), hook_fn)],
            return_type="logits",
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_z_diff[layer, head_index] = normalize_patched_logit_diff(
            patched_logit_diff
        )

In [ ]:
imshow(
    patched_head_z_diff,
    title="Logit Difference From Patched Head Output",
    labels={"x": "Head", "y": "Layer"},
)

## Decomposing Heads

In [ ]:
patched_head_v_diff = torch.zeros(
    model.cfg.n_layers, model.cfg.n_heads, device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_vector, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("v", layer, "attn"), hook_fn)],
            return_type="logits",
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_v_diff[layer, head_index] = normalize_patched_logit_diff(
            patched_logit_diff
        )

In [ ]:
imshow(
    patched_head_v_diff,
    title="Logit Difference From Patched Head Value",
    labels={"x": "Head", "y": "Layer"},
)

In [ ]:
head_labels = [
    f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)
]
scatter(
    x=utils.to_numpy(patched_head_v_diff.flatten()),
    y=utils.to_numpy(patched_head_z_diff.flatten()),
    xaxis="Value Patch",
    yaxis="Output Patch",
    caxis="Layer",
    hover_name=head_labels,
    color=einops.repeat(
        np.arange(model.cfg.n_layers), "layer -> (layer head)", head=model.cfg.n_heads
    ),
    range_x=(-0.5, 0.5),
    range_y=(-0.5, 0.5),
    title="Scatter plot of output patching vs value patching",
)

In [ ]:
def patch_head_pattern(
    corrupted_head_pattern: Float[torch.Tensor, "batch head_index query_pos d_head"],
    hook,
    head_index,
    clean_cache,
):
    corrupted_head_pattern[:, head_index, :, :] = clean_cache[hook.name][
        :, head_index, :, :
    ]
    return corrupted_head_pattern

def ablate_head(
    head_output: Float[torch.Tensor, "batch pos head_index d_head"],
    hook,
    head_index,
):
    head_output[:, :, head_index, :] = 0
    return head_output

def ablate_layer(
    layer_output: Float[torch.Tensor, "batch pos d_model"],
    hook,
    layer_index,
):
    if hook.layer() == layer_index:
        layer_output[:, :, :] = 0
    return layer_output

patched_head_attn_diff = torch.zeros(
    model.cfg.n_layers, model.cfg.n_heads, device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_pattern, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("attn", layer, "attn"), hook_fn)],
            return_type="logits",
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_attn_diff[layer, head_index] = normalize_patched_logit_diff(
            patched_logit_diff
        )

In [ ]:
imshow(
    patched_head_attn_diff,
    title="Logit Difference From Patched Head Pattern",
    labels={"x": "Head", "y": "Layer"},
)
head_labels = [
    f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)
]
scatter(
    x=utils.to_numpy(patched_head_attn_diff.flatten()),
    y=utils.to_numpy(patched_head_z_diff.flatten()),
    hover_name=head_labels,
    xaxis="Attention Patch",
    yaxis="Output Patch",
    title="Scatter plot of output patching vs attention patching",
)

In [ ]:
def run_with_ablated_head(head_index, layer):
    return model.run_with_hooks(
        corrupted_tokens,
        fwd_hooks=[(utils.get_act_name("z", layer, "attn"), partial(ablate_head, head_index=head_index))],
        return_type="logits",
    )

def run_with_ablated_layer(layer_index):
    return model.run_with_hooks(
        corrupted_tokens,
        fwd_hooks=[(utils.get_act_name("resid_post", layer_index), partial(ablate_layer, layer_index=layer_index))],
        return_type="logits",
    )

In [ ]:
# Hypothesis 1: Patching only the early-layer moral salience heads
early_layer_heads = [6*12+2, 5*12+0]  # heads 6H2 and 5H0
for layer, head_index in early_layer_heads:
    hook_fn = partial(patch_head_vector, head_index=head_index, clean_cache=cache)
    patched_logits = model.run_with_hooks(
        corrupted_tokens,
        fwd_hooks=[(utils.get_act_name("z", layer, "attn"), hook_fn)],
        return_type="logits",
    )
    patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)
    print(f"Patching head {layer}H{head_index} gives logit diff: {patched_logit_diff:.3f}")

# Hypothesis 2: Ablating head 9H6
ablated_9H6_logits = run_with_ablated_head(6, 9)
ablated_9H6_logit_diff = logits_to_ave_logit_diff(ablated_9H6_logits, answer_tokens)
print(f"Ablating head 9H6 gives logit diff: {ablated_9H6_logit_diff:.3f}")

# Hypothesis 3: Patching only heads 10H7 and 10H6
negative_heads = [10*12+7, 10*12+6]
for layer, head_index in negative_heads:
    hook_fn = partial(patch_head_vector, head_index=head_index, clean_cache=cache)
    patched_logits = model.run_with_hooks(
        corrupted_tokens,
        fwd_hooks=[(utils.get_act_name("z", layer, "attn"), hook_fn)],
        return_type="logits",
    )
    patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)
    print(f"Patching head {layer}H{head_index} gives logit diff: {patched_logit_diff:.3f}")

# Hypothesis 4: Comparing "either/or" heads on moral vs non-moral prompts
moral_prompts = ["Help make a moral choice. When given the choice to assist either a rich person or a poor person, the right thing to do is to help the"]
non_moral_prompts = ["Help make a moral choice. When given the choice to assist either a nice person or a nice person, the right thing to do is to help the"]

moral_tokens = model.to_tokens(moral_prompts, prepend_bos=True)
_, moral_cache = model.run_with_cache(moral_tokens)

non_moral_tokens = model.to_tokens(non_moral_prompts, prepend_bos=True)
_, non_moral_cache = model.run_with_cache(non_moral_tokens)

either_or_heads = [8*12+11, 8*12+7, 9*12+11]

print("Moral prompts:")
HTML(visualize_attention_patterns(either_or_heads, moral_cache, moral_tokens, layer=8, title="Attention patterns on moral prompts"))

print("Non-moral prompts:")
HTML(visualize_attention_patterns(either_or_heads, non_moral_cache, non_moral_tokens, layer=8, title="Attention patterns on non-moral prompts"))